In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
df1=pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
df1 = df1.rename(index=str, columns={"Agency Type": "AgencyType"})
df1 = df1.rename(index=str, columns={"Distribution Channel": "DistributionChannel"})
df1 = df1.rename(index=str, columns={"Product Name": "ProductName"})
df1 = df1.rename(index=str, columns={"Net Sales": "NetSales"})
df1 = df1.rename(index=str, columns={"Commision (in value)": "Commision"})
df1.drop('ID',axis=1,inplace=True)
test= test.rename(index=str, columns={"Agency Type": "AgencyType"})
test = test.rename(index=str, columns={"Distribution Channel": "DistributionChannel"})
test = test.rename(index=str, columns={"Product Name": "ProductName"})
test = test.rename(index=str, columns={"Net Sales": "NetSales"})
test = test.rename(index=str, columns={"Commision (in value)": "Commision"})

print(df1.columns)
test.columns


Index(['Agency', 'AgencyType', 'DistributionChannel', 'ProductName',
       'Duration', 'Destination', 'NetSales', 'Commision', 'Age', 'Claim'],
      dtype='object')


Index(['ID', 'Agency', 'AgencyType', 'DistributionChannel', 'ProductName',
       'Duration', 'Destination', 'NetSales', 'Commision', 'Age'],
      dtype='object')

In [55]:
df1.describe()

,Duration,NetSales,Commision,Age,Claim
count,52310.000000,52310.000000,52310.000000,52310.000000,52310.000000
mean,58.256108,48.554673,12.219963,39.555725,0.166699
std,109.138708,60.198589,22.847645,13.762473,0.372710
min,-2.000000,-389.000000,0.000000,0.000000,0.000000
25%,10.000000,19.800000,0.000000,33.000000,0.000000
50%,24.000000,29.500000,0.000000,36.000000,0.000000
75%,57.000000,55.000000,13.380000,43.000000,0.000000
max,4881.000000,682.000000,262.760000,118.000000,1.000000


In [56]:
df1.shape

(52310, 10)

In [57]:
categorical_col = ['Agency', 'AgencyType', 'DistributionChannel', 'ProductName', 'Destination']
for col in categorical_col:
    uniq = len(df1[col].unique())
    print(f'{col} :{uniq} Categories')

Agency :16 Categories
AgencyType :2 Categories
DistributionChannel :2 Categories
ProductName :25 Categories
Destination :97 Categories


In [58]:
df1.loc[df1['Duration'] < 0, 'Duration'] = 58.26
df1.loc[df1['Age'] > 100, 'Age'] = 39.56

In [59]:
cat = df1.select_dtypes('object')
con = df1.select_dtypes(exclude='object')
con.head()

,Duration,NetSales,Commision,Age,Claim
0,61.0,12.0,0.00,41.0,0
1,4.0,17.0,0.00,35.0,0
2,26.0,19.8,11.88,47.0,0
3,15.0,27.0,0.00,48.0,0
4,15.0,37.0,0.00,36.0,0


In [60]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [61]:
for i in list(cat.columns):
    cat[str(i)] = le.fit_transform(cat[i])
    test.loc[:,i] = le.transform(test[i])

C:\Users\HaiderAli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [62]:
df1 = pd.concat([con,cat],axis=1)
len(df1)

52310

In [63]:
df1

,Duration,NetSales,Commision,Age,Claim,Agency,AgencyType,DistributionChannel,ProductName,Destination
0,61.0,12.000000,0.0000,41.0,0,7,1,1,10,68
1,4.0,17.000000,0.0000,35.0,0,7,1,1,10,53
2,26.0,19.800000,11.8800,47.0,0,6,1,1,16,84
3,15.0,27.000000,0.0000,48.0,0,7,1,1,1,33
4,15.0,37.000000,0.0000,36.0,0,7,1,1,1,53
...,...,...,...,...,...,...,...,...,...,...
52305,365.0,444.750593,111.1858,40.0,1,2,0,1,3,75
52306,8.0,23.000000,9.2000,54.0,0,12,0,1,24,12
52307,38.0,61.100000,15.2800,37.0,0,2,0,1,9,75
52308,2.0,10.000000,0.0000,46.0,0,7,1,1,10,84


In [64]:
from sklearn.model_selection import train_test_split
from collections import Counter

X = df1.drop('Claim',axis=1)
y = df1.Claim

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=45,test_size=0.2)
X_train.shape,X_test.shape,y_train.shape,y_test.shape


((41848, 9), (10462, 9), (41848,), (10462,))

In [65]:
print('Train shape %s' % Counter(y_train))
print('Test  shape %s' % Counter(y_test))

Train shape Counter({0: 34926, 1: 6922})
Test  shape Counter({0: 8664, 1: 1798})


In [66]:
!pip install imblearn
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_smote, y_smote = sm.fit_resample(X_train, y_train)
print('Resampled Train dataset shape %s' % Counter(y_smote))



Resampled Train dataset shape Counter({0: 34926, 1: 34926})


In [43]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

clf = GradientBoostingClassifier(learning_rate=0.3,n_estimators=2000,max_depth=4)
clf.fit(X_smote,y_smote)

y_pred = clf.predict(X_test)
print("Training Score :",clf.score(X_smote,y_smote))
clf.score(X_test,y_test)

KeyboardInterrupt: 

In [ ]:
gbc_ct = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Prediction'], margins=True)
gbc_ct

In [19]:
1627/(1627+241)

0.8709850107066381

In [20]:
precision = gbc_ct[1][1]/(gbc_ct[1][1]+gbc_ct[1][0])
precision


0.8893258426966292

In [21]:
from sklearn import ensemble

rf = ensemble.RandomForestClassifier()
rf.fit(X_smote, y_smote)
y_pred = rf.predict(X_test)
print("Training Score :",clf.score(X_smote,y_smote))
clf.score(X_test,y_test)

Training Score : 0.9935005440073298


0.9606193844389218

In [22]:
rf_ct = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Prediction'], margins=True)
rf_ct

Prediction,0,1,All
Actual,,,
0,8337,327,8664
1,250,1548,1798
All,8587,1875,10462


In [23]:
precision = rf_ct[1][1]/(rf_ct[1][1]+rf_ct[1][0])
precision

0.8256

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report , accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score , roc_curve,precision_score
from sklearn.preprocessing import StandardScaler, Normalizer,LabelEncoder
from xgboost import XGBClassifier

In [68]:
xgbclf = XGBClassifier(scale_pos_weight=0.1)
xgbclf.fit(X_smote,y_smote)
y_pred = xgbclf.predict(X_test)
precision_score(y_test,y_pred)

0.9440559440559441

In [69]:
xgb_ct = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Prediction'], margins=True)
xgb_ct

Prediction,0,1,All
Actual,,,
0,8624,40,8664
1,1123,675,1798
All,9747,715,10462


In [47]:
precision = xgb_ct[1][1]/(xgb_ct[1][1]+xgb_ct[1][0])
precision

0.9440559440559441

In [22]:
print("Training Score :",clf.score(X_smote,y_smote))
clf.score(X_test,y_test)

AttributeError: 'NoneType' object has no attribute 'tree_'

In [70]:
testid = test.ID
test.drop("ID",axis=1,inplace=True)

In [76]:
print(df1.columns)


Index(['Duration', 'NetSales', 'Commision', 'Age', 'Claim', 'Agency',
       'AgencyType', 'DistributionChannel', 'ProductName', 'Destination'],
      dtype='object')


In [81]:
print(df1.columns)
test = test.reindex(columns=['Duration', 'NetSales', 'Commision', 'Age', 'Agency',
       'AgencyType', 'DistributionChannel', 'ProductName', 'Destination'])

Index(['Duration', 'NetSales', 'Commision', 'Age', 'Claim', 'Agency',
       'AgencyType', 'DistributionChannel', 'ProductName', 'Destination'],
      dtype='object')


In [82]:
test_pred = xgbclf.predict(test)

In [84]:
sub = pd.DataFrame({"ID":testid,"Claim":test_pred})
sub.head()

,ID,Claim
0,17631,0
1,15064,0
2,14139,0
3,19754,0
4,16439,0


In [87]:
sub.to_csv('submission.csv',index=False)